In [34]:
import matplotlib as plt
print('matplotlib: {}'.format(plt.__version__))
# pandas
import pandas as pd
print('pandas: {}'.format(pd.__version__))
# scikit-learn
import sklearn as sk
print('sklearn: {}'.format(sk.__version__))
from pandas_profiling import ProfileReport
import seaborn as sns
print('seaborn: {}'.format(sns.__version__))
import numpy as np
print('numpy: {}'.format(np.__version__))
from sklearn import linear_model
from sklearn import svm
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

matplotlib: 3.3.2
pandas: 1.1.3
sklearn: 0.23.2
seaborn: 0.11.0
numpy: 1.19.2


In [2]:
X_train = pd.read_csv("X_train.csv")

In [3]:
X_val=pd.read_csv("X_val.csv")
X_test = pd.read_csv("X_test.csv")

In [4]:
y_train = pd.read_csv("y_train.csv")
y_val = pd.read_csv("y_val.csv")
y_test = pd.read_csv("y_test.csv")

In [5]:
id_test = pd.read_csv("id_test.csv")

In [15]:
X_train = X_train.drop(columns=["Unnamed: 0"])

In [17]:
X_val = X_val.drop(columns=["Unnamed: 0"])

In [19]:
X_test = X_test.drop(columns=["Unnamed: 0"])

In [21]:
y_train =y_train.drop(columns=["Unnamed: 0"])

In [23]:
y_val = y_val.drop(columns=["Unnamed: 0"])

In [25]:
y_test = y_test.drop(columns=["Unnamed: 0"])

In [27]:
id_test = id_test.drop(columns=["Unnamed: 0"])

In [36]:
y_test.sample()

,SalePrice_Log
120,12.100712


In [35]:
models = []
models.append(('LR', LinearRegression()))
models.append(('Ridge', linear_model.Ridge()))
models.append(('SVR', svm.SVR()))
models.append(('Lasso',linear_model.Lasso()))
models.append(('ElasticNet',ElasticNet()))
results = []
names = []
for name, model in models:
    kfold = KFold(n_splits=10, random_state=1, shuffle=True)
    cv_results = cross_val_score(model, X_train, y_train, cv=kfold,scoring='neg_root_mean_squared_error')
    results.append(cv_results)
    names.append(name)
    print('%s: %f (%f)' % (name, cv_results.mean(), cv_results.std()))

LR: -0.135828 (0.008469)
Ridge: -0.134779 (0.007722)


C:\Users\dalil\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\dalil\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\dalil\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\dalil\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**

SVR: -0.336435 (0.017211)
Lasso: -0.302402 (0.020140)
ElasticNet: -0.299317 (0.020669)


In [ ]:
# The best model is the ridge model